In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE48352'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"DPYS as a potential diagnostic biomarker in hereditary and sporadic PRCC2"
!Series_summary	"Papillary renal cell carcinoma type 2 (PRCC2) is known to be very aggressive type of tumor without effictive therapy. Hereditary form of PRCC2 is caused by Fumarate Hydratase (FH) gene mutation that accompanied Hereditary Leiomyomatosis and Renal Cell Carcinoma (HLRCC) disorder. In sporadic form of PRCC2 the mutation of FH gene has not been reported. Both forms of tumors have the similar histopathological characteristics with poor survival prognosis."
!Series_summary	"In this study, we profiled the gene expression of renal tumors and normal tissue from PRCC2 (hereditary and sporadic) patients in order to better understand commonalities and differences in the transcriptional landscape of PRCC2."
!Series_overall_design	"Microarray gene expression profiling was performed on eight normal kidney tissue samples, five hereditary PRCC2 tumor tissue samples and 19 s

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Analyze the dataset information

# Check if gene expression data is available
is_gene_available = True  # since it states "gene expression profiling"

# Determine the keys in the sample characteristics dictionary
# Based on the provided dictionary:
# 0: ['tissue type: Normal', 'tissue type: Cancer'] doesn't seem to represent age or gender
# 1: ['disease status: hereditary PRCC2', 'disease status: sporadic PRCC2'] relates to the trait

trait_row = 1
age_row = None  # No information about age is given
gender_row = None  # No information about gender is given

# Define conversion functions
def convert_trait(value):
    try:
        _, v = value.split(': ')
        return 1 if 'PRCC2' in v else 0
    except Exception:
        return None

convert_age = lambda x: None  # since age data isn't available
convert_gender = lambda x: None  # since gender data isn't available

# Save cohort information
save_cohort_info('GSE48352', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE48352.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1176286': [1], 'GSM1176287': [1], 'GSM1176288': [1], 'GSM1176289': [1], 'GSM1176290': [1], 'GSM1176291': [1], 'GSM1176292': [1], 'GSM1176293': [1], 'GSM1176294': [1], 'GSM1176295': [1], 'GSM1176296': [1], 'GSM1176297': [1], 'GSM1176298': [1], 'GSM1176299': [1], 'GSM1176300': [1], 'GSM1176301': [1], 'GSM1176302': [1], 'GSM1176303': [1], 'GSM1176304': [1], 'GSM1176305': [1], 'GSM1176306': [1], 'GSM1176307': [1], 'GSM1176308': [1], 'GSM1176309': [1], 'GSM1176310': [1], 'GSM1176311': [1], 'GSM1176312': [1], 'GSM1176313': [1], 'GSM1176314': [1], 'GSM1176315': [1], 'GSM1176316': [1], 'GSM1176317': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}


### Step 6: Gene Identifier Mapping

In [7]:
# Given that `requires_gene_mapping` is True, proceed with the substeps.

# 1. Define the keys for identifier and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Description'

# 2. Obtain the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE48352.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# 4. Save the cohort information.
save_cohort_info('GSE48352', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE48352.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Kidney_Papillary_Cell_Carcinoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is severely biased.

